In [50]:
import numpy as np
import pandas as pd
import cv2

import os 
import fnmatch
import keras

In [51]:
#Reading sensor tested data in csv
Sensor_tested_data = pd.read_csv("C:\\Users\\Mahmood Yousaf\\Desktop\\Final-Year-Project-Soil-Analysis-using-machine-learning\\Source Code\\Machine Learning\\Deployed models testing\\PreprocessedSensorValues.csv")
print(Sensor_tested_data.iloc[:,4])

0      0.666667
1      0.800000
2      0.800000
3      0.725429
4      0.800000
         ...   
138    0.950000
139    0.750000
140    0.750000
141    0.500000
142    0.800000
Name: EC, Length: 143, dtype: float64


In [52]:
sampleID = Sensor_tested_data.iloc[:, 1]
EC = Sensor_tested_data.iloc[:, 4]
P = Sensor_tested_data.iloc[:, 3]
pH = Sensor_tested_data.iloc[:, 2]

In [53]:
def load_images_from_folder(folder):
    images = {}
    image_counter = 0
    for filename in os.listdir(folder):
        x = filename.split("_")
        id = int(x[0])
        img = cv2.imread(os.path.join(folder,filename))
        width = 1000
        height = 1000
        dim = (width, height)
        # resize image
        img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        img = img[500:800,500:800] #Resizing the image
        kernel = np.array([[-1,-1,-1], 
                   [-1, 9,-1],
                   [-1,-1,-1]])
        img = cv2.filter2D(img, -1, kernel) # applying the sharpening kernel.
        Result = sampleID.isin([id])
        Result = sampleID[Result]
        #Checking that there should be sample id exists
        if len(Result) >= 1:
            if id in images.keys():
                continue
            else:
                images[id] = img
                print(image_counter)
                image_counter += 1
        #Checking there is no mactch than continue the looping 
        else:
            continue
    return images

In [54]:
dataset = load_images_from_folder('E:\\FYP\\Dataset RGB\\Mixed_data')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [55]:
dataset.keys()

dict_keys([110639, 110645, 110660, 110661, 110662, 110667, 110737, 110740, 110746, 110752, 110755, 110760, 110761, 110762, 110768, 110857, 110868, 110869, 110872, 110873, 110902, 110988, 110989, 120910, 122722, 122723, 122736, 122934, 122935, 122938, 122940, 122941, 123047, 123060, 123062, 123065, 123572, 123573, 123579, 123580, 123583, 123678, 123680, 123698, 123701, 123707, 123709, 123710, 123711, 123718, 123719, 123722, 123723, 123911, 123918, 123921, 123922, 123923, 123930, 123939, 123940, 123949, 123950, 123956, 123964, 123976, 123979, 123981, 123992, 123998, 124000, 124002, 124003, 124006, 124007, 124008, 124185, 124188, 124189, 124190, 124200, 124201, 124202, 124205, 124206, 124208, 124216, 124226, 124228, 124229, 124232, 124233, 124234, 124236, 124238, 124239, 124240, 124241, 124243, 124244])

In [56]:
import pickle
Pmodel = pickle.load(open('Pclassifier.pkl', 'rb'))
pHmodel = pickle.load(open('pHclassifier.pkl', 'rb'))
ECmodel = pickle.load(open('ECclassifier.pkl', 'rb'))

In [57]:
results = {}
for key in dataset:
    width = 1000
    height = 1000
    dim = (width, height)
    # resize image
    img = cv2.resize(dataset[key], dim, interpolation = cv2.INTER_AREA)
    img = img[500:800,500:800] #Resizing the image
    kernel = np.array([[-1,-1,-1], 
               [-1, 9,-1],
               [-1,-1,-1]])
    img = cv2.filter2D(img, -1, kernel) # applying the sharpening kernel.
    # extracting blue,red,green channel from color image
    blue_channel = img[:,:,0]
    green_channel = img[:,:,1]
    red_channel = img[:,:,2]
    temp = ((np.median(green_channel)+np.median(blue_channel))+np.median(red_channel))
    temp = np.nanmean(temp)
    Presult = float(Pmodel.predict([[temp]]))
    pHreuslt = float(pHmodel.predict([[temp]]))
    ECresult = float(ECmodel.predict([[temp]]))
    tempResult = {'P':Presult, 'pH':pHreuslt, 'EC':ECresult } 
    results[key] = tempResult
    print(tempResult)

{'P': 5.050000000000007, 'pH': 7.782116666666658, 'EC': 1.36}
{'P': 4.965000000000005, 'pH': 7.759449999999992, 'EC': 1.14}
{'P': 4.985028571428571, 'pH': 7.714692857142867, 'EC': 0.7933333333333334}
{'P': 5.050000000000007, 'pH': 7.762116666666658, 'EC': 1.36}
{'P': 5.07951832889333, 'pH': 7.668764646138182, 'EC': 0.94}
{'P': 4.691058477633477, 'pH': 7.629163780663789, 'EC': 1.1233333333333333}
{'P': 4.721166666666664, 'pH': 7.74538333333334, 'EC': 1.13}
{'P': 4.957000000000005, 'pH': 7.759449999999992, 'EC': 1.14}
{'P': 5.122999999999999, 'pH': 7.748666666666659, 'EC': 1.05}
{'P': 6.255064285714285, 'pH': 7.777999999999994, 'EC': 0.825}
{'P': 4.31051585081585, 'pH': 7.780739960039962, 'EC': 1.0885714285714287}
{'P': 6.255064285714285, 'pH': 7.777999999999994, 'EC': 0.825}
{'P': 5.122999999999999, 'pH': 7.610666666666663, 'EC': 1.02}
{'P': 4.964133333333337, 'pH': 7.785666666666673, 'EC': 0.56}
{'P': 5.262034523809525, 'pH': 7.721133333333338, 'EC': 1.42}
{'P': 4.7655222222222235, 'pH

In [58]:
print(results)

{110639: {'P': 5.050000000000007, 'pH': 7.782116666666658, 'EC': 1.36}, 110645: {'P': 4.965000000000005, 'pH': 7.759449999999992, 'EC': 1.14}, 110660: {'P': 4.985028571428571, 'pH': 7.714692857142867, 'EC': 0.7933333333333334}, 110661: {'P': 5.050000000000007, 'pH': 7.762116666666658, 'EC': 1.36}, 110662: {'P': 5.07951832889333, 'pH': 7.668764646138182, 'EC': 0.94}, 110667: {'P': 4.691058477633477, 'pH': 7.629163780663789, 'EC': 1.1233333333333333}, 110737: {'P': 4.721166666666664, 'pH': 7.74538333333334, 'EC': 1.13}, 110740: {'P': 4.957000000000005, 'pH': 7.759449999999992, 'EC': 1.14}, 110746: {'P': 5.122999999999999, 'pH': 7.748666666666659, 'EC': 1.05}, 110752: {'P': 6.255064285714285, 'pH': 7.777999999999994, 'EC': 0.825}, 110755: {'P': 4.31051585081585, 'pH': 7.780739960039962, 'EC': 1.0885714285714287}, 110760: {'P': 6.255064285714285, 'pH': 7.777999999999994, 'EC': 0.825}, 110761: {'P': 5.122999999999999, 'pH': 7.610666666666663, 'EC': 1.02}, 110762: {'P': 4.964133333333337, 'p

In [59]:
IDs = []
for key in results:
    IDs.append(key)

In [60]:
P = []
pH = []
EC = []
for value in results.values():
    P.append(value['P'])
    pH.append(value['pH'])
    EC.append(value['EC'])

In [61]:
P = pd.Series(P, name='P')
pH = pd.Series(pH, name='pH')
EC = pd.Series(EC, name='EC')
IDs = pd.Series(IDs, name='sampleID')

In [62]:
predictions = pd.concat([IDs, P, pH, EC], axis=1)

In [63]:
predictions.to_csv("ImagePredictionsResults.csv")